# Create Daily Heists

In [1]:
import os, sys, random, json
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.utils import timezone
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import emporium.logic.guild as GD
import emporium.logic.stage as ST
import emporium.logic.simulation as SM
print([x for x in dir(EM) if x[0]!='_' and len(x)>8])

['AdvancedRoom', 'BasicRoom', 'CastleRoom', 'ExpeditionLevel', 'ExpeditionType', 'GothicTower', 'LeagueTrial', 'RequiredPower', 'RoomUpgrade', 'ThiefLevel', 'UniqueRoom', 'UnlockableItem', 'UnlockableThief']


In [7]:
import engine.models as GM
import engine.logic.content as CT
import engine.logic.launcher as LH
import engine.logic.resource as RS
import engine.logic.robot as RB
print([x for x in dir(GM) if x[0]!='_' and len(x)>8])

['GuildExpedition', 'GuildStage', 'ItemInGuild', 'ItemUnlocked', 'MarketStore', 'RoomInGuild', 'ThiefInGuild', 'ThiefUnlocked']


In [8]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Development

In [9]:
# get user

userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [25]:
# view stages for all guilds

stageLs = GM.GuildStage.objects.filter().values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'id', 'BackgroundBias', 
                                     'ObstaclesL2', 'ObstaclesL3', 'ObstaclesL4', 'ObstaclesL5',], axis=1, errors='ignore')
print( len(stageLs) )

groupDf = stageDf[['GuildFK_id', 'Heist', 'StageNo']]
groupDf = groupDf.groupby(['GuildFK_id', 'Heist']).agg(['count']).reset_index()
groupDf.columns = [' '.join(str(c) for c in col) for col in groupDf.columns]
groupDf

0


KeyError: "None of [Index(['GuildFK_id', 'Heist', 'StageNo'], dtype='object')] are in the [columns]"

In [23]:
# get selected guild

guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)
PD.DataFrame([guildMd.__dict__]).drop(['_state', 'UserFK_id'], axis=1, errors='ignore')

,id,Name,LastPlayed,CreateDate,Selected,VaultGold,VaultStone,VaultGems,LastHeist,CampaignWorld,DungeonCheck
0,25,963,2025-01-08,2025-01-07,True,133,49,23,dungeon,1,None


In [19]:
# reset stages

GM.GuildStage.objects.filter(GuildFK=guildMd).delete()
guildMd.DungeonCheck = None
guildMd.save()

stageLs = GM.GuildStage.objects.filter(GuildFK=guildMd).values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'GuildFK_id'], axis=1, errors='ignore')
stageDf

(12, {'engine.GuildStage': 12})

""


In [13]:
# reset thieves

thiefLs = GM.ThiefInGuild.objects.filter(GuildFK=guildMd)
for th in thiefLs:
    th.Status = 'Ready'
    th.save()
print('thieves reset')

thieves reset


In [20]:
stageLs = GM.GuildStage.objects.filter(GuildFK=guildMd).values()
stageDf = PD.DataFrame(stageLs).drop(['_state', 'id', 'GuildFK_id', 'BackgroundBias', 
                                     'ObstaclesL2', 'ObstaclesL3', 'ObstaclesL4', 'ObstaclesL5',], axis=1, errors='ignore')
len(stageLs)
stageDf[12:15]

0

""
